In [226]:
import pandas as pd
import datetime
import time
import math

In [227]:
pd.set_option('float_format', '{:f}'.format)
folder = '03_01_2019_00_48_WF_p'
res_adapt = pd.read_csv(folder + '/adapt.csv')
res_adapt_eval = pd.read_csv(folder + '/adapt_eval.csv')
res_eval = pd.read_csv(folder + '/eval.csv')

In [228]:
res_adapt.drop(res_adapt[res_adapt.instance.str.contains('exact')].index, inplace=True)
res_eval.drop(res_eval[res_eval.instance.str.contains('exact')].index, inplace=True)
res_adapt_eval.drop(res_adapt_eval[res_adapt_eval.instance.str.contains('exact')].index, inplace=True)

In [229]:
res_corr = res_eval.merge(res_adapt, left_on='instance', right_on='instance', how='inner')

In [230]:
res_corr = res_corr.fillna(0)
res_corr['diff_p'] = (res_corr['pred_p'] - res_corr['real_p'])
res_corr['diff_p'] = res_corr['diff_p']*res_corr['diff_p']

In [231]:
df1 = res_corr[res_corr['type_x'] == 'FeatureBased_SVM']['pred_p'].reset_index()
df2 = res_corr[res_corr['type_x'] == 'CRNN']['cos'].reset_index()
pd.concat([df1, df2], axis=1).corr().loc['pred_p', 'cos']

-0.024848813729872132

In [234]:
res_corr.groupby('type_x')[['pred_p', 'old_p', 'old_r', 'old_f', 'cos', 'sum']].corr()

pred_p     old_p  \
type_x                                                                  
CRNN                                       pred_p  1.000000  0.176024   
                                           old_p   0.176024  1.000000   
                                           old_r   0.576204  0.309938   
                                           old_f   0.106215  0.335372   
                                           cos    -0.025623  0.091040   
                                           sum     0.039865 -0.012628   
DNN                                        pred_p  1.000000  0.150684   
                                           old_p   0.150684  1.000000   
                                           old_r   0.519951  0.309938   
                                           old_f   0.116549  0.335372   
                                           cos    -0.031064  0.091040   
                                           sum     0.049971 -0.012628   
FeatureBased_Bayesian Ridge Regression     pred_p  1.000000  0.054316   
                                           old_p   0.054316  1.000000   
                                           old_r   0.242353  0.309938   
                                           old_f   0.062347  0.335372   
                                           cos          nan       nan   
                                           sum          nan       nan   
FeatureBased_Lasso Regression              pred_p  1.000000  0.054316   
                                           old_p   0.054316  1.000000   
                                           old_r   0.242353  0.309938   
                                           old_f   0.062347  0.335372   
                                           cos          nan       nan   
                                           sum          nan       nan   
FeatureBased_Linear Regression             pred_p  1.000000  0.054316   
                                           old_p   0.054316  1.000000   
                                           old_r   0.242353  0.309938   
                                           old_f   0.062347  0.335372   
                                           cos          nan       nan   
                                           sum          nan       nan   
FeatureBased_Passive Aggressive Regression pred_p  1.000000 -0.011400   
                                           old_p  -0.011400  1.000000   
                                           old_r   0.076065  0.309938   
                                           old_f  -0.046928  0.335372   
                                           cos          nan       nan   
                                           sum          nan       nan   
FeatureBased_SGD Regression                pred_p  1.000000 -0.017208   
                                           old_p  -0.017208  1.000000   
                                           old_r   0.047424  0.309938   
                                           old_f  -0.051697  0.335372   
                                           cos          nan       nan   
                                           sum          nan       nan   
FeatureBased_SVM                           pred_p  1.000000  0.054316   
                                           old_p   0.054316  1.000000   
                                           old_r   0.242353  0.309938   
                                           old_f   0.062347  0.335372   
                                           cos          nan       nan   
                                           sum          nan       nan   
FeatureBased_Theil-Sen Regression          pred_p  1.000000 -0.011684   
                                           old_p  -0.011684  1.000000   
                                           old_r   0.074720  0.309938   
                                           old_f  -0.047178  0.335372   
                                           cos          nan       nan   
                                      

In [235]:
res_adapt['diff_p'] = res_adapt['new_p'] - res_adapt['old_p']
res_adapt['diff_r'] = res_adapt['new_r'] - res_adapt['old_r']
res_adapt['diff_f'] = res_adapt['new_f'] - res_adapt['old_f']

In [236]:
res_adapt['pair'], res_adapt['alg'] = res_adapt['instance'].str.split(' ', 1).str
res_adapt2 = res_adapt[~(res_adapt['alg'].isin(['AMC Token Path', 'WordNet Jiang Conrath', 'Ontobuilder Term Match']))]

In [238]:
res_adapt.groupby('type').mean()

,Unnamed: 0,k,old_p,old_r,old_f,new_p,new_r,new_f,diff_p,diff_r,diff_f
type,,,,,,,,,,,
BPR,2840,0,0.502087,0.562980,0.388842,0.460080,0.514424,0.348884,-0.042006,-0.048556,-0.039958
CRNN,2842,0,0.502087,0.562980,0.388842,0.618812,0.590787,0.599051,0.116725,0.027806,0.210209
DNN,2841,0,0.502087,0.562980,0.388842,0.621020,0.582402,0.594116,0.118933,0.019422,0.205275
IR,2838,0,0.502087,0.562980,0.388842,0.494615,0.500000,0.497287,-0.007472,-0.062980,0.108445
MULTI,2843,0,0.502087,0.562980,0.388842,0.618476,0.589158,0.597844,0.116389,0.026178,0.209002
SVDpp,2839,0,0.502087,0.562980,0.388842,0.502465,0.503734,0.502325,0.000378,-0.059246,0.113483


In [239]:
res_adapt2.groupby('type').mean()

,Unnamed: 0,k,old_p,old_r,old_f,new_p,new_r,new_f,diff_p,diff_r,diff_f
type,,,,,,,,,,,
BPR,2829.634981,0.000000,0.502024,0.562308,0.388915,0.459992,0.514246,0.349470,-0.042031,-0.048062,-0.039445
CRNN,2831.634981,0.000000,0.502024,0.562308,0.388915,0.619702,0.590507,0.598813,0.117679,0.028199,0.209898
DNN,2830.634981,0.000000,0.502024,0.562308,0.388915,0.621094,0.581869,0.593615,0.119071,0.019561,0.204700
IR,2827.634981,0.000000,0.502024,0.562308,0.388915,0.494614,0.500000,0.497286,-0.007409,-0.062308,0.108371
MULTI,2832.634981,0.000000,0.502024,0.562308,0.388915,0.618825,0.589003,0.597520,0.116801,0.026695,0.208605
SVDpp,2828.634981,0.000000,0.502024,0.562308,0.388915,0.502921,0.504036,0.502607,0.000897,-0.058272,0.113692


In [255]:
sums = res_eval[['instance', 'sum', 'pred_p']][res_eval['type'].isin(['DNN', 'FeatureBased_SVM'])]
res_adapt = res_adapt.merge(sums, left_on='instance', right_on='instance', how='inner')
res_adapt['pair'], res_adapt['alg'] = res_adapt['instance'].str.split(' ', 1).str

In [260]:
res_adapt_max = res_adapt.sort_values('sum', ascending=True).drop_duplicates(['pair'])
res_adapt_max.groupby('type').mean()[['old_p', 'old_r', 'old_f']]

,old_p,old_r,old_f
type,,,
BPR,0.501937,0.503821,0.339610
CRNN,0.497902,0.439267,0.383208
DNN,0.499950,0.502585,0.388761
IR,0.499646,0.493100,0.380239
MULTI,0.498596,0.487145,0.395916
SVDpp,0.500699,0.507181,0.389278


In [258]:
res_adapt_max = res_adapt.sort_values('pred_p', ascending=False).drop_duplicates(['pair'])
res_adapt_max.groupby('type').mean()[['old_p', 'old_r', 'old_f']]

,old_p,old_r,old_f
type,,,
BPR,0.502192,0.529563,0.385119
CRNN,0.502187,0.558404,0.390035
DNN,0.501324,0.586310,0.384737
IR,0.501229,0.545025,0.385972
MULTI,0.498845,0.501002,0.378782
SVDpp,0.501559,0.535635,0.363416


In [246]:
res_adapt_eval['final_p'] = res_adapt_eval[['old_p', 'new_p']].max(axis=1)
res_adapt_eval['final_r'] = res_adapt_eval[['old_r', 'new_r']].max(axis=1)
res_adapt_eval['final_f'] = res_adapt_eval[['old_f', 'new_f']].max(axis=1)

In [247]:
# idx = res_adapt_eval.groupby([['instance', 'type']])['k'].transform(max) == df['k']
# best_k = res_adapt_eval[idx]
res_adapt_eval_k = res_adapt_eval.sort_values('final_f', ascending=False).drop_duplicates(['instance', 'type'])

In [248]:
res_adapt_eval_k.groupby('type').mean()[['final_p', 'final_r', 'final_f']]

,final_p,final_r,final_f
type,,,
CRNN_CRNN,0.621516,0.607597,0.599145
MULTI,0.620683,0.603527,0.597923


In [249]:
res_adapt_eval_k['pair'], res_adapt_eval_k['alg'] = res_adapt_eval_k['instance'].str.split(' ', 1).str
res_adapt_eval_max = res_adapt_eval_k.sort_values('pred_e', ascending=False).drop_duplicates(['pair'])
res_adapt_eval_max.groupby('type').mean()[['final_p', 'final_r', 'final_f']]

,final_p,final_r,final_f
type,,,
CRNN_CRNN,0.965312,0.860169,0.904343
MULTI,0.953346,0.866141,0.901692


In [105]:
res_adapt_eval[res_adapt_eval['type'] =='MULTI'].groupby('pair').max()[['final_p', 'final_r', 'final_f']].mean()

final_p   0.779419
final_r   0.826513
final_f   0.774038
dtype: float64

In [102]:
res_adapt['new_f_max'] = res_adapt.groupby(['pair', 'type'])['new_f'].transform(max)
clean_a = res_adapt[res_adapt['new_f_max'] == res_adapt['new_f']]
clean_a.groupby('type').mean()
# res_adapt['diff_f_max'] = res_adapt.groupby(['pair'])['diff_f'].transform(max)
# clean_a = res_adapt[res_adapt['diff_f_max'] == res_adapt['diff_f']]
# clean_a.groupby('type').mean()
# clean_a.groupby('type').mean()
# grouped = clean_a[clean_a['type'] == 'CNN'].groupby(['type', 'pair']).mean().reset_index()
# grouped.sort_values('diff_f', ascending=False)

KeyError: 'pair'

In [133]:
CRNN_adapt = res_adapt[res_adapt['type'] =='CRNN']

In [134]:
CRNN_adapt['new_f_max'] = CRNN_adapt.groupby(['pair'])['new_f'].transform(max)

C:\Users\shrag\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [135]:
CRNN_adapt_max = CRNN_adapt[CRNN_adapt['new_f_max'] == CRNN_adapt['new_f']]

In [137]:
CRNN_adapt_max.mean()

Unnamed: 0   3964.323232
k               0.000000
old_p           0.651197
old_r           0.750034
old_f           0.646826
new_p           0.733820
new_r           0.713100
new_f           0.705204
diff_p          0.082623
diff_r         -0.036934
diff_f          0.058378
pair                 inf
new_f_max       0.705204
dtype: float64

In [138]:
res_adapt_eval['pred_e_max'] = res_adapt_eval.groupby(['instance','type'])['real_e'].transform(max)
res_adapt_eval['k_max'] = res_adapt_eval.groupby(['instance','type'])['k'].transform(max)
clean_a_e = res_adapt_eval[(res_adapt_eval['pred_e_max'] == res_adapt_eval['real_e']) &
               (res_adapt_eval['k_max'] == res_adapt_eval['k'])]

In [139]:
res_adapt_eval

,Unnamed: 0,instance,type,k,old_p,old_r,old_f,new_p,new_r,new_f,real_e,pred_e,pred_e_max,k_max
0,0,1 AMC Token Path Ontobuilder Max Delta,GRU_GRU,0,0.510861,0.756943,0.463253,0.512335,0.768540,0.473108,0.463253,0.458160,0.463253,1
1,1,1 AMC Token Path Ontobuilder Max Delta,GRU_GRU,1,0.512335,0.768540,0.473108,0.512335,0.768540,0.473108,0.463253,0.680636,0.463253,1
2,2,1 AMC Token Path Ontobuilder Max Delta,CNN_CNN,0,0.510861,0.756943,0.463253,0.496379,0.499669,0.498018,0.463253,0.899688,0.463253,1
3,3,1 AMC Token Path Ontobuilder Max Delta,CNN_CNN,1,0.496379,0.499669,0.498018,0.496379,0.499669,0.498018,0.463253,0.967738,0.463253,1
4,4,1 AMC Token Path Ontobuilder Max Delta,DNN_DNN,0,0.510861,0.756943,0.463253,0.496382,0.500000,0.498184,0.463253,0.674669,0.463253,0
5,5,1 AMC Token Path Ontobuilder Max Delta,CRNN_CRNN,0,0.510861,0.756943,0.463253,0.496360,0.497018,0.496689,0.463253,0.300762,0.463253,1
6,6,1 AMC Token Path Ontobuilder Max Delta,CRNN_CRNN,1,0.496360,0.497018,0.496689,0.496360,0.497018,0.496689,0.463253,0.814879,0.463253,1
7,7,1 AMC Token Path Ontobuilder Max Delta,GRU_CNN,0,0.510861,0.756943,0.463253,0.512335,0.768540,0.473108,0.463253,0.899688,0.463253,0
8,8,1 AMC Token Path Ontobuilder Max Delta,GRU_DNN,0,0.510861,0.756943,0.463253,0.512335,0.768540,0.473108,0.463253,0.674669,0.463253,1
9,9,1 AMC Token Path Ontobuilder Max Delta,GRU_DNN,1,0.512335,0.768540,0.473108,0.512335,0.768540,0.473108,0.463253,0.704076,0.463253,1


In [140]:
clean_a_e.groupby('type').mean()[['new_p', 'new_r', 'new_f']]

,new_p,new_r,new_f
type,,,
BPR_FEATBayesian Ridge Regression,0.510357,0.572868,0.489646
BPR_FEATLasso Regression,0.510357,0.572868,0.489646
BPR_FEATLinear Regression,0.510357,0.572868,0.489646
BPR_FEATPassive Aggressive Regression,0.510357,0.572868,0.489646
BPR_FEATSGD Regression,0.510357,0.572868,0.489646
BPR_FEATSVM,0.510357,0.572868,0.489646
BPR_FEATTheil-Sen Regression,0.510357,0.572868,0.489646
BPR_GRU,0.510357,0.572868,0.489646
CNN_CNN,0.544619,0.554299,0.535674


In [141]:
res_adapt_eval['pair'], res_adapt_eval['alg'] = res_adapt_eval['instance'].str.split(' ', 1).str
CRNN_adapt_eval = res_adapt_eval[res_adapt_eval['type'] =='CRNN']

In [20]:
res_adapt['pair'], res_adapt['alg'] = res_adapt['instance'].str.split(' ', 1).str

In [142]:
CRNN_adapt_eval['new_f_max'] = CRNN_adapt_eval.groupby(['pair'])['new_f'].transform(max)

In [143]:
CRNN_adapt_max = CRNN_adapt[CRNN_adapt['new_f_max'] == CRNN_adapt['new_f']]

In [146]:
CRNN_adapt_max.mean()

Unnamed: 0   3964.323232
k               0.000000
old_p           0.651197
old_r           0.750034
old_f           0.646826
new_p           0.733820
new_r           0.713100
new_f           0.705204
diff_p          0.082623
diff_r         -0.036934
diff_f          0.058378
pair                 inf
new_f_max       0.705204
dtype: float64

In [145]:
def summerize(data, file):
    idMax = data[data['type'] == data['type']]
    if 'pred_e' in data:
        idMax = data.groupby(['type'])['pred_e'].transform(max) == data['pred_e']
        pd.DataFrame(data[idMax].groupby(by='type').mean()).to_csv(file)
    else:
        pd.DataFrame(data.groupby(by='type').mean()).to_csv(file)

In [23]:
data = pd.read_csv('adapt.csv', low_memory=False)
summerize(data, 'a.csv')

FileNotFoundError: File b'adapt.csv' does not exist